In [5]:
from bs4 import BeautifulSoup

In [32]:
corpus = {}

with open("nos.html", "r") as page: 
    texts = []
    tok_count = 0
    doc_count = 0
    soup = BeautifulSoup(page, "html.parser")
    content = soup.find_all("div", class_="itemcontent")
    for piece in content:
        texts.append(piece.get_text())
        tok_count += len(piece.get_text().split(" "))

    corpus["tokens"] = tok_count
    corpus["docs"] = len(texts)
    corpus["texts"] = texts

In [33]:
corpus

{'tokens': 7573,
 'docs': 20,
 'texts': ['In Nieuwe Niedorp in Noord-Holland is begonnen met het opgraven van een bommenwerper uit de Tweede Wereldoorlog.\nHet vliegtuig werd in de nacht van 22 op 23 juni 1941 uit de lucht \ngeschoten door een Duitse nachtjager, toen het terugvloog naar Engeland \nna een bombardement op Bremen. Het gaat om een Vickers Wellington van de\n Britse Royal Air Force (RAF). Er waren zes Tsjechische bemanningsleden \naan boord.\n"Het toestel is hier in de Kostverlorenpolder de bodem ingegaan", \nweet majoor Bart Aalberts van de Luchtmacht. Hij is projectleider van de\n berging. "We hebben veel overleg gehad met de eigenaren van het terrein\n en via detectie hebben we uiteraard gekeken of het vliegtuig hier ook \nligt."\nSinds deze week is een graafmachine bezig om voorzichtig de eerste \nlagen grond weg te schrapen. De zoektocht moet duidelijkheid geven over \nde vijf bemanningsleden die bij de crash om het leven kwamen. \nWaarschijnlijk liggen hun stoffelijke

In [36]:
import pickle

with open('nos.pickle', 'wb') as handle:
    pickle.dump(corpus, handle)